In [7]:
pip install pandas

In [2]:
from langdetect import detect
from tqdm.notebook import tqdm
import pandas as pd
from tqdm.notebook import tqdm
import pandas as pd



0it [00:00, ?it/s]

## Filtering english text only

In [3]:
data=pd.read_csv("data_all.csv")

In [4]:
data.head(5)
len(data)

3955

In [ ]:
# Rename the columns 
data=data.rename(columns = {'content':'Text'})
data=data.drop(columns=["score","embedding","content_type","id","meta"])

data.head(5)

In [11]:
data['lang'] = data.Text.astype(str).progress_map(detect)


  0%|          | 0/3955 [00:00<?, ?it/s]

In [15]:
data.lang.value_counts()

lang
en    3680
Name: count, dtype: int64

In [16]:
data = data.loc[data.lang=='en']

In [17]:
data.columns = map(str.lower, data.columns)

data.head(6)

,text,country,lang
0,published by the general secretariat of the na...,20201231_NDC_Update_Cambodia.docx,en
1,the updated nationally determined contribution...,20201231_NDC_Update_Cambodia.docx,en
2,finally a significant effort has been made to ...,20201231_NDC_Update_Cambodia.docx,en
3,say samal minister of environment chair of the...,20201231_NDC_Update_Cambodia.docx,en
4,together we agreed to limit global warming to ...,20201231_NDC_Update_Cambodia.docx,en
5,cambodia is highly vulnerable to the effects o...,20201231_NDC_Update_Cambodia.docx,en


In [18]:
#data.to_csv(r'C:\\Users\\serva\\Downloads\\data_all.csv', index = True)

In [29]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

In [24]:

documents=data["text"]

documents.head(10)



0    published by the general secretariat of the na...
1    the updated nationally determined contribution...
2    finally a significant effort has been made to ...
3    say samal minister of environment chair of the...
4    together we agreed to limit global warming to ...
5    cambodia is highly vulnerable to the effects o...
6    business as usual bau emission scenarios in th...
7    summary over bau emissions and ndc emissions r...
8    impacts and priorities for adaptation and resi...
9    such actions can be divided into education act...
Name: text, dtype: object

In [30]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

costumized_stopwords=["united kingdom","argentina","australia","bahamas","belgium","brazil","canada","china","colombia","cuba","dominican republic","ecuador","el salvador","france","germany","guatemala","haiti","honduras","india","ireland","israel","italy","japan","korea","mexico","netherlands","philippines","spain","sweden","switzerland","taiwan","venezuela","viet nam","afghanistan","albania","algeria","american samoa","andorra","angola","armenia","austria","azerbaijan","bahrain","bangladesh","barbados","belarus","belize","benin","bermuda","bhutan","bolivia","bosnia and herzegovina","botswana","bulgaria","burkina faso","burundi","cambodia","cameroon","cape verde islands","chad","chile","congo","costa rica","croatia","cyprus","czech republic","denmark","djibouti","dominica","egypt","eritrea","estonia","ethiopia","fiji","finland","french polynesia","gabon","gambia","georgia","ghana","greece","grenada","guinea","guyana","hungary","iceland","indonesia","iran","iraq","jamaica","jordan","kazakhstan","kenya","korea","kuwait","latvia","lebanon","liberia","libya","lithuania","luxembourg","madagascar","malawi","malaysia","maldives","mali","malta","mauritania","mauritius","monaco","mongolia","montenegro","morocco","mozambique","namibia","nepal","new zealand","nicaragua","niger","nigeria","norway","oman","pakistan","panama","papua new guinea","paraguay","peru","poland","portugal","qatar","romania","rwanda","saudi arabia","senegal","serbia","sierra leone","singapore","slovakia","slovenia","solomon islands","somalia","south africa","sri lanka","sudan","suriname","swaziland","tajikistan","thailand","togo","trinidad and tobago","tunisia","turkey","turkmenistan","tuvalu","uganda","ukraine","united arab emirates","united states","uruguay","uzbekistan","vanuatu","yemen","zambia"
]

""""stpwrd = gensim.parsing.preprocessing.STOPWORDS and costumized_stopwords """


stpwrd = nltk.corpus.stopwords.words('english')
stpwrd.extend(costumized_stopwords)



def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))



def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in stpwrd and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\serva\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [31]:
stemmer = SnowballStemmer('english')
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in original_words]
pd.DataFrame(data = {'original word': original_words, 'stemmed': singles})

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [33]:
import nltk
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\serva\AppData\Roaming\nltk_data...


True

In [34]:

processed_docs = data['text'].map(preprocess)



In [39]:
processed_docs.head(5)

0    [publish, general, secretariat, nation, counci...
1    [updat, nation, determin, contribut, present, ...
2    [final, signific, effort, make, develop, solid...
3    [samal, minist, environ, chair, nation, counci...
4    [togeth, agre, limit, global, warm, well, pref...
Name: text, dtype: object

In [49]:
import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel

from pprint import pprint

import spacy

import pickle
import re 
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

import matplotlib.pyplot as plt 
import pandas as pd

# Bigrams and Trigrams


In [50]:

data_words=processed_docs
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['publish', 'general', 'secretariat', 'nation', 'council', 'sustain', 'environ', 'kingdom', 'general', 'secretariat', 'nation', 'council', 'sustain', 'environ', 'kingdom', 'right', 'reserv', 'part', 'public', 'reproduc', 'transmit', 'form', 'mean', 'without', 'permiss', 'copyright', 'holder', 'contact', 'inform', 'general', 'secretariat', 'nation', 'council', 'sustain', 'environ', 'morodok', 'techo', 'build', 'tonl', 'bassac', 'chamkarmon', 'phnom_penh', 'updat', 'nation', 'determin', 'contribut', 'foreword', 'intern', 'communiti', 'reach', 'truli', 'histor', 'mileston', 'pari', 'climat', 'agreement', 'togeth', 'agre', 'limit', 'global', 'warm', 'well', 'prefer', 'degre_celsius', 'compar', 'preindustri_level', 'signific', 'reduc', 'risk', 'impact', 'climat', 'chang', 'alway', 'strong', 'support', 'multilater', 'action', 'climat', 'chang', 'countri', 'particular', 'vulner', 'impact', 'climat', 'chang', 'understand', 'urgenc', 'ambiti', 'climat', 'action', 'lead', 'exampl', 'line', 'capa

In [51]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]


In [52]:
# Form Bigrams
data_words_trigrams = make_trigrams(data_words)



In [53]:
# Create Dictionary 
id2word = corpora.Dictionary(data_words )  
id2word.filter_extremes(no_below=10, no_above=0.2)
# Create Corpus 
texts = data_words 
# Term Document Frequency 
corpus = [id2word.doc2bow(text) for text in texts]  
# View 
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 3), (9, 1), (10, 3), (11, 1), (12, 1), (13, 1), (14, 1), (15, 3), (16, 1), (17, 1), (18, 2), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 3), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1)]]


In [54]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('agre', 1),
  ('agreement', 1),
  ('alway', 1),
  ('ambiti', 1),
  ('celsius', 1),
  ('communiti', 1),
  ('compar', 1),
  ('convent', 1),
  ('council', 3),
  ('degre', 1),
  ('environ', 3),
  ('exampl', 1),
  ('foreword', 1),
  ('form', 1),
  ('framework', 1),
  ('general', 3),
  ('global', 1),
  ('histor', 1),
  ('kingdom', 2),
  ('lead', 1),
  ('limit', 1),
  ('line', 1),
  ('mean', 1),
  ('mileston', 1),
  ('multilater', 1),
  ('pari', 1),
  ('part', 1),
  ('particular', 1),
  ('prefer', 1),
  ('preindustri', 1),
  ('public', 1),
  ('publish', 1),
  ('reach', 1),
  ('reserv', 1),
  ('right', 1),
  ('risk', 1),
  ('secretariat', 3),
  ('signific', 1),
  ('strong', 1),
  ('togeth', 1),
  ('understand', 1),
  ('unfccc', 1),
  ('unit', 1),
  ('urgenc', 1),
  ('warm', 1),
  ('well', 1),
  ('without', 1)]]

In [55]:
lda_model = gensim.models.ldamodel.LdaModel(
corpus=corpus, # Der corpus mit BoW oder TF-IDF
id2word=id2word, # id2word gibt das Dictionary aus den Wörtern wieder
num_topics=10, # Die mitgegebene Topic Anzahl
random_state=100, # Das spezifische Seed, damit der Prozess wieder gleich wiedergegeben werden kann
update_every=1,# Gibt an wie oft das Modell geupdatet werden soll
chunksize=100, # Die Anzahl an Dokumenten die auf einmal angeschaut wird
passes=10,# Wie oft der Algorithmus durchlaufen wird
alpha='auto', # Das vorgegebene Gewicht Alpha
per_word_topics=True)

In [56]:
# Print the keyword of topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.029*"temperatur" + 0.023*"loss" + 0.018*"damag" + 0.018*"extrem" + '
  '0.017*"flood" + 0.016*"event" + 0.016*"affect" + 0.016*"popul" + '
  '0.015*"rainfal" + 0.015*"rise"'),
 (1,
  '0.024*"growth" + 0.023*"global" + 0.021*"economi" + 0.017*"import" + '
  '0.017*"effort" + 0.015*"challeng" + 0.015*"signific" + 0.013*"continu" + '
  '0.012*"popul" + 0.012*"commit"'),
 (2,
  '0.053*"forest" + 0.048*"land" + 0.033*"ecosystem" + 0.029*"product" + '
  '0.023*"south" + 0.023*"protect" + 0.023*"coastal" + 0.021*"marin" + '
  '0.018*"biodivers" + 0.017*"food"'),
 (3,
  '0.042*"financ" + 0.039*"institut" + 0.035*"technolog" + 0.030*"fund" + '
  '0.024*"privat" + 0.023*"remov" + 0.023*"applic" + 0.018*"public" + '
  '0.018*"invest" + 0.017*"financi"'),
 (4,
  '0.042*"wast" + 0.041*"transport" + 0.032*"effici" + 0.032*"electr" + '
  '0.027*"industri" + 0.025*"renew" + 0.022*"generat" + 0.015*"sourc" + '
  '0.015*"product" + 0.015*"vehicl"'),
 (5,
  '0.029*"articl" + 0.025*"palestin" + 

In [57]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.131310 -0.120403       1        1  16.787574
1      0.054981 -0.270336       2        1  14.106287
5      0.245957 -0.053942       3        1  14.059751
8      0.080690  0.122999       4        1  11.702369
4     -0.047811  0.048264       5        1  10.947409
3      0.101448  0.245408       6        1   8.949469
2     -0.176607  0.100257       7        1   8.581165
0     -0.132012 -0.199292       8        1   8.215159
7      0.063099  0.113532       9        1   4.195142
9     -0.321057  0.013514      10        1   2.455674, topic_info=          Term         Freq        Total Category  logprob  loglift
190     forest  1146.000000  1146.000000  Default  30.0000  30.0000
98        wast  1157.000000  1157.000000  Default  29.0000  29.0000
74        land  1036.000000  1036.000000  Default  28.0000  28.0000
252  transport  1120.000000  1120.000000  Default  27.0000  27.0000
75    ministri   777.000000   777.000000  Default  26.0000  26.0000
..         ...          ...          ...      ...      ...      ...
722      solut    78.799374   221.437025  Topic10  -4.3582   2.6735
229      renew    78.453338   773.059006  Topic10  -4.3626   1.4189
239      small    59.955566   330.927546  Topic10  -4.6315   1.9985
732    biomass    50.570246   117.626588  Topic10  -4.8018   2.8626
368      locat    51.898779   186.811216  Topic10  -4.7758   2.4260

[541 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
348       4  0.994471    abil
349       2  0.011677  access
349       4  0.319959  access
349       5  0.042038  access
349       6  0.623569  access
...     ...       ...     ...
140       2  0.831809   would
140       4  0.016504   would
261       6  0.996950   youth
1051      5  0.942706    zero
1051      7  0.054650    zero

[840 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 2, 6, 9, 5, 4, 3, 1, 8, 10])

In [58]:
# Save the visualization in a html file
p = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
pyLDAvis.save_html(p, 'lda_visualization.html')

In [60]:
pip install wordcloud

  Obtaining dependency information for wordcloud from https://files.pythonhosted.org/packages/34/ac/72a4e42e76bf549dfd91791a6b10a9832f046c1d48b5e778be9ec012aa47/wordcloud-1.9.2-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/151.4 kB ? eta -:--:--
   ---------------------------------------- 0.0/151.4 kB ? eta -:--:--
   ---------------------------------------- 0.0/151.4 kB ? eta -:--:--
   ---------- ----------------------------- 41.0/151.4 kB ? eta -:--:--
   ---------- ----------------------------- 41.0/151.4 kB ? eta -:--:--
   ---------- ----------------------------- 41.0/151.4 kB ? eta -:--:--
   --------------- ----------------------- 61.4/151.4 kB 469.7 kB/s eta 0:00:01
   --------------- ----------------------- 61.4/151.4 kB 469.7 kB/s eta 0:00:01
   --------------- ----------------------- 61.4/151.4 kB 469.7 kB/s eta 0:00:01
   --------------- ----------------------- 61.4/151.4 kB 469.7 kB/s eta 0:00:01
   ---------------------------- -------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [65]:
from wordcloud import WordCloud
from os import path
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os

lda=lda_model




wc = WordCloud(background_color="white", max_words=1000, mask=alice_mask)
# generate word cloud
wc.generate_from_frequencies(WordCloud().fit_words(dict(lda.show_topic(t, 200))
                                                   
for t in range(10):
    wc = WordCloud(background_color="white", max_words=1000, mask=alice_mask)
    wc.generate_from_frequencies(WordCloud().fit_words(dict(lda.show_topic(t, 200))
    # show
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.show()
    plt.title("Topic " + str(t))
    
wc.generate(text)
for t in range(10):
    plt.imshow(wc, interpolation='bilinear')
    plt.figure()
    plt.imshow(WordCloud().fit_words(dict(lda.show_topic(t, 200))))
    plt.axis("off")
    plt.title("Topic " + str(t))
    plt.show() 

SyntaxError: invalid syntax (3179373350.py, line 17)

In [63]:
# Creating Wordclouds for every Topic 

"""wc.generate(text)
for t in range(10):
    plt.figure()
    plt.imshow(WordCloud(max_font_size=100, max_words=100, background_color="white").fit_words(dict(lda.show_topic(t, 200))))
    plt.axis("off")
    plt.title("Topic " + str(t))
    plt.show()
    WordCloud(max_font_size=100, max_words=100, background_color="white").fit_words(dict(lda.show_topic(t, 200))).to_file("first_review.png")"""


'wc.generate(text)\nfor t in range(10):\n    plt.figure()\n    plt.imshow(WordCloud(max_font_size=100, max_words=100, background_color="white").fit_words(dict(lda.show_topic(t, 200))))\n    plt.axis("off")\n    plt.title("Topic " + str(t))\n    plt.show()\n    WordCloud(max_font_size=100, max_words=100, background_color="white").fit_words(dict(lda.show_topic(t, 200))).to_file("first_review.png")'

In [66]:

import os

# Change the current working directory
os.chdir('C:\\Users\\serva\\Downloads')
#Adding image
chicken_mask = np.array(Image.open("chicken.jpg"))
tree_mask = np.array(Image.open("Euro.png"))
#Path for the writing style
font_path = 'font.otf'

# lower max_font_size, change the maximum number of word and lighten the background:
wordcloud = WordCloud(
max_font_size=200, max_words=100, background_color="white",font_path=font_path,mask=chicken_mask).fit_words(dict(lda.show_topic(0, 150)))

plt.figure()
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

# Save the image in the img folder:
wordcloud.to_file("Chicken_pic.png")

FileNotFoundError: [Errno 2] No such file or directory: 'chicken.jpg'

# Coherence Score

In [64]:
data_lemmatized=data_words
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  
# a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.944260699010695

Coherence Score:  0.4764790286274829
